In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,AutoModelForSequenceClassification,AutoConfig
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

In [15]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

In [16]:

def calculate_metrics(true_labels, predicted_labels):
    
    true_labels_flat = [label for sublist in true_labels for label in sublist]
    predicted_labels_flat = [label for sublist in predicted_labels for label in sublist]
    
    #because manual labelling and different tokenizer can can generate different length token lists 
    minlen = min(len(true_labels_flat),len(predicted_labels_flat))
    true_labels_flat = true_labels_flat[:minlen]
    predicted_labels_flat = predicted_labels_flat[:minlen]
    
    precision = precision_score(true_labels_flat, predicted_labels_flat, average='weighted',zero_division=1)
    recall = recall_score(true_labels_flat, predicted_labels_flat, average='weighted',zero_division=1)

    f1 = f1_score(true_labels_flat, predicted_labels_flat, average='weighted',zero_division=1)
    macro_f1 = f1_score(true_labels_flat, predicted_labels_flat, average='macro',zero_division=1)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)
    print("Macro F1: ", macro_f1)

    return precision, recall, f1, macro_f1

# Loading the saved IndicNER model

In [17]:

# config = AutoConfig.from_pretrained('/Users/abinash/Desktop/ComputationalLinguistics/Assignment2/IndicNER/')
model_ner = AutoModelForTokenClassification.from_pretrained('./IndicNER/')
tokenizer_ner = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")


# Loading the IndicBERT model

In [18]:
# config_bert = AutoConfig.from_pretrained('./IndicBERT/')
tokenizer_bert = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model_bert = AutoModelForTokenClassification.from_pretrained('./IndicBERT/', num_labels=7 )

# Loading the sentenses from Question 1

In [19]:
with open('sentences.txt', 'r') as file:
    sentences = file.readlines()

print(sentences)
#remove the newline character from the sentences
sentences = [sentence.strip() for sentence in sentences]

print(sentences)


['उन्होंने आज ट्वीट किया, ‘क्या किसी किसी पीटी ने प्रस्तावित जीएसटी संविधान संशोधन में जीएसटीएन संभावनाओं और भूमिका का अध्ययन किया है।\n', 'कुमुद महाजन का कहना है कि वर्तमान समय की भागदौड़ वाली जीवनशैली के कारण लोगों में सिरदर्द\xa0\n', 'मोदी सरकार ने गाय के लिए भी बजट में विशेष तवज्जो दी है.\n', "शायद हमें कभी ना पता चले लेकिन यह बहुत दुखद घटना है।' खबर में बताया गया है कि दोनों 2014 से शादीशुदा थे और दोनों सॉफ्टवेयर इंजीनियर थे। \n", 'फरवरी में पुलवामा आतंकवादी हमले के बाद भारत और पाकिस्तान के संबंधों में कटुता आने के कारण पाकिस्तानी नौसेना ने अपने मित्र देश की नौसेना के महत्वपूर्ण आयोजन में भाग नहीं लिया.\n', 'इस बात की जानकारी फ्लॉरेडा के शेरिफ ऑफिस ने बताई.\n', 'मुख्यमंत्री ने कहा कि सभी गांवों को सड़कों से जोड़ना है।\n', '2-3 किलोमीटर की परिधि में फैले इस छोटे से इलाके ने मलक्का के 600 सालों के इतिहास को समेट रखा है.\n', 'इसलिए मैंने शुरू में कहा था इंदिरा इज इंडिया, जो नारा उस समय कांग्रेस ने लगाया था, कांग्रेस अध्यक्ष ने लगाया था और जिसका परिणाम 1977 में देखने को मिला.\n', 'इस दौ

Manually written NER Tags

In [20]:
with open('truth.txt', 'r') as file:
    truth = file.readlines()


print(truth)
#remove the newline character from the truth
truth = [sentence.strip() for sentence in truth]
print(truth)

true_labels = [sentence.split() for sentence in truth]
print(true_labels)



['O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'B-PER I-PER O O O O O O O O O O O O O O O\n', 'B-ORG I-ORG O B-MISC O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O B-LOC O O O O B-LOC O B-LOC O O O O O O O B-ORG I-ORG O O O O O B-ORG O O O O O O O\n', 'O O O O B-LOC O B-ORG B-ORG O O O\n', 'B-PER O O O O O O O O O O O\n', 'O O O O O O O O O O O O O B-LOC O O O O O O O O O O\n', 'O O O O O O B-PER O O O O O O O B-ORG O O O O B-ORG O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O O O\n', 'B-LOC O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O\n', 'B-PER I-PER O B-PER I-PER O O O O O O O O O O O O O O O O O O O O O O O B-PER O O O O O O O\n', 'B-MISC I-MISC I-MISC I-MISC O O O O O O O O\n', 'O O O O O O O O O O O O O \n', 'O O O B-ORG I-ORG O B-PER I-PER O B-PER O O O O O O O O O O

# Comparing IndicNER with manually written NER tags

In [21]:
# predicing the named entities using the NER model
predicted_labels_ner = []
for sentence in sentences:
    predicted_labels_ner.append(get_predictions(sentence, tokenizer_ner, model_ner))

print(calculate_metrics(true_labels, predicted_labels_ner))

Precision:  0.9308647652834692
Recall:  0.9274047186932849
F1:  0.9580580464240267
Macro F1:  0.5160007297938333
(0.9308647652834692, 0.9274047186932849, 0.9580580464240267)


# Comparing IndicBERT with manually written NER tags

In [22]:
# predicing the named entities using the BERT model
predicted_labels_bert = []
for sentence in sentences:
    predicted_labels_bert.append(get_predictions(sentence, tokenizer_bert, model_bert))

print(calculate_metrics(true_labels, predicted_labels_bert))


Precision:  0.8479116740842176
Recall:  0.8681102362204725
F1:  0.922591599940071
Macro F1:  0.5480385247827109
(0.8479116740842176, 0.8681102362204725, 0.922591599940071)


# Comparing ChatGPT with manually written NER tags

In [23]:
with open('chatgpt.txt', 'r') as file:
    chatgpt_sentences = file.readlines()

print(chatgpt_sentences)

#remove the newline character from the sentences
chatgpt_sentences = [sentence.strip() for sentence in chatgpt_sentences]

print(chatgpt_sentences)

chatgpt_lebel = [sentence.split() for sentence in chatgpt_sentences]
print(chatgpt_lebel)

['O O O O O B-LOC O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'B-PER I-PER O O O O O O O O O O O O O O O O\n', 'B-PER O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O B-LOC O B-LOC O O O O O O O O B-LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O B-LOC O B-ORG I-ORG O O\n', 'B-PER O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O B-LOC O O O O O O\n', 'O O O O B-PER I-PER I-PER O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O O O O O O\n', "'O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n", 'B-LOC O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O O O O O O O O O O O O O O O O\n', 'B-PER I-PER O O O B-PER I-PER O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O\n', 'O O O O O O O O O 

In [24]:

print(calculate_metrics(true_labels, chatgpt_lebel))

Precision:  0.8434016074669433
Recall:  0.8711433756805808
F1:  0.9322678977322785
Macro F1:  0.7702265372168284
(0.8434016074669433, 0.8711433756805808, 0.9322678977322785)


In [25]:
# for i in range(len(sentences)):
#     print(sentences[i], len(sentences[i]))
#     print(true_labels[i], len(true_labels[i]))
#     print(predicted_labels_ner[i], len(predicted_labels_ner[i]))
#     print(predicted_labels_bert[i], len(predicted_labels_bert[i]))
#     print(chatgpt_lebel[i], len(chatgpt_lebel[i]))
#     print("\n\n\n")

In [26]:
# # remove "," "[" "]" from chatgpt_truth usinf regex
# import re
# text = """['O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-ORG', 'I-ORG', 'O', 'O']
# ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['B-LOC', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"""
# text = [re.sub(r'[\[\],]', '', sentence) for sentence in text.split('\n')]
# print(text)

# # remove ' form the text
# text = [sentence.replace("'", "") for sentence in text]
# print(text)